## Sentinel-1 feature tracking

This application takes a pair of Sentinel-1 products and identifies and generates the coherence

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype with the following processing steps:

A. S1 SAR processing per image:

* Application of orbit file (should wait for the orbit file a couple of days, since for coherence this is important)
* TOPS slice assembly (if necessary)
* TOPS split (if necessary)

B. For image pair:

* TOPS coregistration
* Coherence estimation (Given set of images from same orbit,  {t1, t2, t3, ... , tn}, two temporally adjacent images would * constitute an image pair, with the first one being the master. So the pairs would be: {t1 - t2, t2 - t3, ... , tn-1 - tn}.)
* TOPS deburst
* Multi-looking
* Terrain correction


TOPS --> Terrain Observation by Prograssive Scans

TOPS Slice assembly-->assemble subswaths back together

TOPS split -->split into subswaths



### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Sentinel-1 feature tracking'),
                ('abstract', 'Sentinel-1 feature tracking'),
                ('id', 'ewf-ethz-02-02-01')])

In [ ]:
max_velocity = dict([('id', 'max_velocity'),
                     ('value', '8'),
                     ('title', 'Max velocity'),
                     ('abstract', 'Max velocity (m/day)')])

polarisation = dict([('id', 'polarisation'),
                     ('value', 'VV'),
                     ('title', 'Polarisation'),
                     ('abstract', 'Polarisation')])

In [ ]:
aoi = dict([('id', 'area_of_interest'),
            ('value', 'POLYGON((-91.32114821675749 -1.141198173686936, -90.71737318778665 -1.009922494598408, -90.84942543017247 -0.5555272457029629, -91.40848642576378 -0.6637246974332075, -91.32114821675749 -1.141198173686936))'),
            ('title', 'Area of interest in WKT'),
            ('abstract', 'Area of interest in WKT')])

In [ ]:
dem = dict([('id', 'dem'),
            ('value', 'SRTM 3Sec'),
            ('title', 'Area of interest in WKT'),
            ('abstract', 'Area of interest in WKT'),
            ('options', 'SRTM 3Sec,ACE30')])

In [ ]:
resolution = dict([("id", "resolution"),
                   ("title", "Spatial resolution"),
                   ("value", "40"),
                   ("abstract", "Spatial resolution in meters (10 or 40)"),
                   ('options', '10,40')])

In [ ]:
utm_zone = dict([('id', 'utm_zone'),
                 ('title', 'UTM zone'),
                 ('abstract', 'UTM zone'),
                 ('value', 'EPSG:32715')])

In [ ]:
window_size = dict([('id', 'window_size'),
                    ('title', 'window_size'),
                    ('abstract', 'window size in pixels'),
                    ('value', '64')])

In [ ]:
oversampling_factor = dict([('id', 'oversampling_factor'),
                            ('title', 'oversampling_factor'),
                            ('abstract', 'oversampling factor'),
                            ('value', '8')])

In [ ]:
pixel_size = dict([('id', 'pixel_size'),
                   ('title', 'pixel_size'),
                   ('abstract', 'pixel size in meters'),
                   ('value', '40')])

In [ ]:
color_scale_limits = dict([('id', 'color_scale_limits'),
                           ('title', 'color_scale_limits'),
                           ('abstract', 'color_scale_limits'),
                           ('value', '0,10')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [ ]:
input_identifiers = ('S1A_IW_GRDH_1SDV_20190824T002619_20190824T002648_028703_033FCF_961C',
                     'S1A_IW_GRDH_1SDV_20190929T002620_20190929T002649_029228_0351FE_D742')

**Input references**

These are the Sentinel-1 catalogue references

In [ ]:
input_references = ('https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20190824T002619_20190824T002648_028703_033FCF_961C',
                    'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20190929T002620_20190929T002649_029228_0351FE_D742')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ellip_snap_helpers
import xml.etree.ElementTree as ET
from scipy import interpolate
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
import geopandas as gpd
from datetime import datetime

import matplotlib.pyplot as plt

from shapely.geos import ReadingError
import gzip
import shutil
import csv 
import gdal
import osr
import math
import time
import exifread
import lxml.etree as etree
import rasterio

from shapely.wkt import loads
from shapely.geometry import mapping

import numpy as np

from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")

import glob

sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import subprocess
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from osgeo.gdalconst import GA_ReadOnly
from struct import unpack
from PIL import Image
from PIL import ImageDraw
import cioppy
ciop = cioppy.Cioppy()

os.environ['LD_LIBRARY_PATH'] = '/opt/v94/runtime/glnxa64:/opt/v94/bin/glnxa64:/opt/v94/sys/os/glnxa64:/opt/v94/extern/bin/glnxa64'
import run_dic

### AOI

In [ ]:
if aoi['value'] == 'Full':
    aoi_wkt = cascaded_union(search.geometry.values).wkt
    min_lon, min_lat, max_lon, max_lat = cascaded_union(search.geometry.values).bounds

else:

    try:
        aoi_wkt = loads(aoi['value']).wkt
        min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

    except ReadingError:

        aoi_wkt = box(*[float(i) for i in aoi['value'].split(',')]).wkt
        min_lon, min_lat, max_lon, max_lat = [float(i) for i in aoi['value'].split(',')]

In [ ]:
print aoi_wkt

## Read the products

### check if all the products have the same track number

In [ ]:
product_TR = [None]*len(input_references)

for index,product_ref in enumerate(input_references):
    
    result_prod = ciop.search(end_point=product_ref,
                              params=[],
                              output_fields='startdate,track',
                              model='EOP')
    
    product_TR[index] = result_prod[0]['track']
    
    if index==0:
        
        slave_date = result_prod[0]['startdate'][:10]
    
    elif result_prod[0]['startdate'][:10] > slave_date:
    
        slave_date = result_prod[0]['startdate'][:10]

if not all(x == product_TR[0] for x in product_TR):

    raise ValueError('Not all products pertain the same track!')

#### Read the products

In [ ]:
s1meta = "manifest.safe"

#slave_date = ''

slave_products = []
master_products = []

slave_prefix = []
master_prefix = []

dates = []

for index, identifier in enumerate(input_identifiers):
    
    s1_zip_file = os.path.join(data_path, identifier + '.zip') 
    s1_meta_file = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe')

    if os.path.isfile(s1_zip_file):
        s1prd = s1_zip_file
    elif os.path.isfile(s1_meta_file):
        s1prd = s1_meta_file

    print identifier, s1prd
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    
    
    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    start_date = parser.parse(product.getStartTime().toString()).isoformat()
    
    dates.append(start_date[:19])

    if start_date[:10] == slave_date:
        
            slave_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
            slave_prefix.append(identifier.split('_')[-1]) 
            slave_data_take = identifier.split('_')[-2]
    else:
            master_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as master" % (name, width, height, start_date))
            master_data_take = identifier.split('_')[-2]  
            master_prefix.append(identifier.split('_')[-1]) 

                        
output_name = 'S1_OFFSET_TRACKING_%s_%s' % (parser.parse(min(dates)).strftime('%Y%m%d%H%M%S'),
                                                     parser.parse(max(dates)).strftime('%Y%m%d%H%M%S'))

print("\nco-registered OUTPUT Img name is %s"%output_name)


In [ ]:
mygraph = ellip_snap_helpers.GraphProcessor()

#### Read and if need assemble the products

In [ ]:
operator = 'Read'

node_id = 'Read'

source_node_id = ''

In [ ]:
if len(slave_products) > 1:
  
    slave_read_nodes = []
    
    # Read 
    for index, slave_identifier in enumerate(slave_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-S(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = slave_identifier
                
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        slave_read_nodes.append(node_id)
    
    
    source_nodes_id = slave_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-S'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    #parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_slave_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-S'
        
    source_node_id = ''
        
    parameters['file'] = slave_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_slave_orbit = node_id

In [ ]:
if len(master_products) > 1:
      
    master_read_nodes = []
    
    # Read 
    for index, master_identifer in enumerate(master_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-M(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = master_identifer
        
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        master_read_nodes.append(node_id)
    
    
    source_nodes_id = master_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-M'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    #parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_master_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-M'
        
    source_node_id = ''
        
    parameters['file'] = master_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_master_orbit = node_id

In [ ]:
mygraph.view_graph()

### Apply orbit file

In [ ]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-S' 

source_node_id = source_slave_orbit

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['orbitType'] = 'Sentinel Restituted (Auto Download)'

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-M' 

source_node_id = source_master_orbit

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
mygraph.view_graph()

### Land/sea mask

In [ ]:
operator = 'Land-Sea-Mask'

node_id = 'Land-Sea-Mask-S' 

source_node_id = 'Apply-Orbit-File-S' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['landMask'] = 'false'

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
operator = 'Land-Sea-Mask'

node_id = 'Land-Sea-Mask-M' 

source_node_id = 'Apply-Orbit-File-M' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['landMask'] = 'false'

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### DEM assisted coregistration

In [ ]:
operator = 'DEM-Assisted-Coregistration'

node_id = 'DEM-Assisted-Coregistration' 

source_node_id = ['Land-Sea-Mask-S',
                  'Land-Sea-Mask-M']

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### Subset

In [ ]:
operator = 'Subset'

node_id = 'Subset' 

source_node_id = 'DEM-Assisted-Coregistration'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['geoRegion'] = aoi_wkt
parameters['copyMetadata'] = 'true'

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### Terrain correction

In [ ]:
operator = 'Terrain-Correction'

source_node_id = 'Subset' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['demName'] = dem['value']
parameters['pixelSpacingInMeter'] = resolution['value']
parameters['mapProjection'] = utm_zone['value']

node_id = 'Terrain-Correction'

In [ ]:
parameters

In [ ]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### Write

In [ ]:
operator = 'Write'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['file'] = output_name
parameters['formatName'] = 'GeoTIFF-BigTIFF'

node_id = 'Write'

source_node_id = 'Terrain-Correction'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
mygraph.view_graph()

In [ ]:
mygraph.run()

### Separate tiffs

In [ ]:
output_tif = '{}.tif'.format(output_name)
f = open(output_tif, 'rb')
tags = exifread.process_file(f)
xml_data = tags['Image OwnerName'].values
tree = ET.XML(xml_data)

metadata = dict()

for child in tree.find('Image_Interpretation'):
    band_index = child.find('BAND_INDEX').text
    name = child.find('BAND_NAME').text
    metadata[band_index] = name
    
metadata

In [ ]:
print gdal.Info("./S1_OFFSET_TRACKING_20190824002619_20190929002620.tif")

In [ ]:
src = gdal.Open("./S1_OFFSET_TRACKING_20190824002619_20190929002620.tif")

geo_transform = src.GetGeoTransform()
projection = src.GetProjection()

for band_number in range(1, src.RasterCount+1):
     
    band_data = src.GetRasterBand(band_number).ReadAsArray()
    band_description = metadata[str(band_number-1)]
    
    band_data = np.where(band_data==0, np.nan, band_data)
    
    print band_description

    imgplot = plt.imshow(band_data)
    plt.show()
    
    drv = gdal.GetDriverByName('GTiff')
    ds = drv.Create('{}.tif'.format(band_description), band_data.shape[1], band_data.shape[0], 1, gdal.GDT_Float32)
    ds.SetGeoTransform(geo_transform)
    ds.SetProjection(projection)
    ds.GetRasterBand(1).WriteArray(band_data)
    ds.FlushCache()

#### List of geotiffs produced

In [ ]:
master_list = list()
slave_list = list()

for element in metadata.values():
    if 'mst' in element:
        master_list.append(element)
    if 'slv' in element:
        slave_list.append(element)

### run_dic

#### writing the input_dic.txt

In [ ]:
input_list = list()

for idx, master_tif in enumerate(sorted(slave_list)):
    
    input_list.append('input_dic_{}.txt'.format(idx))

    with open('input_dic_{}.txt'.format(idx), 'wb') as file:
        file.write('{}.tif\n'.format(master_tif))
        file.write('{}.tif\n'.format(sorted(master_list)[idx]))
        file.write('{} {} {}\n'.format(window_size['value'],oversampling_factor['value'], pixel_size['value']))
        file.write('{} {}\n'.format(color_scale_limits['value'].split(',')[0], color_scale_limits['value'].split(',')[1]))

#### running the package 

In [ ]:
os.environ['LD_LIBRARY_PATH'] = '/opt/v94/runtime/glnxa64:/opt/v94/bin/glnxa64:/opt/v94/sys/os/glnxa64:/opt/v94/extern/bin/glnxa64'

for input_file in input_list:
    
    print input_file
    
    command = 'import run_dic; mr = run_dic.initialize(); mr.run_dic(\"{}\", nargout=0)'.format(input_file)

    options = ['python', '-c', command]
    
    p = subprocess.Popen(options,
                         stdout=subprocess.PIPE,
                         stdin=subprocess.PIPE,
                         stderr=subprocess.PIPE)

    res, err = p.communicate()

    if res:
        print 'RESULTS:\n'
        for el in res.split('\n'):
            print el

    if err:
        print 'ERRORS:\n'
        for el in res.split('\n'):
            print el

    print

In [ ]:
os.remove(output_tif)

for tif in master_list:
    os.remove('{}.tif'.format(tif))
    
for tif in slave_list:
    os.remove('{}.tif'.format(tif))
    
for input_file in input_list:
    os.remove(input_file)

In [ ]:
print gdal.Info('./VV_VV_2d_offset_bg.tiff')

In [ ]:
for file in os.listdir('./'):
    if '.tif' in file:
        print file
        with rasterio.open(file, 'r') as ds:
            arr = ds.read()
            
        drv = gdal.GetDriverByName('GTiff')
        ds = drv.Create('{}.tif'.format(os.path.splitext(os.path.basename(file))[0]), arr.shape[2], arr.shape[1], arr.shape[0], gdal.GDT_Byte)
        ds.SetGeoTransform(geo_transform)
        ds.SetProjection(projection)
        for band_number in range(arr.shape[0]):
            ds.GetRasterBand(band_number+1).WriteArray(arr[band_number])
        ds.FlushCache()
        
        os.remove(file)

In [ ]:
print gdal.Info('./VV_VV_2d_offset_bg.tif')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.